In [ ]:
!pip install jdaviz==4.5.0

In [ ]:
# Aperture photometry
# Download overlapping JWST images of omega Centauri from NIRCam and MIRI.

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from jdaviz import Imviz
from regions import CircleAnnulusSkyRegion, CircleSkyRegion

In [ ]:
# viewport size parameter (in pixels)
# (600, 1000, 2000, 4000) pixels per side
viewport_pixel_size = {viewport_pixel_size_value}  # noqa: F821

# compress images generated in the python kernel, with this method before sending to the client
# ('png', 'webp', 'none')
image_compression = "{image_compression_value}"  # noqa: F821

# use gwcs_to_fits_sip while loading the data
# (True, False)
gwcs_to_fits_sip = {gwcs_to_fits_sip_value}  # noqa: F821

# number of subsets
# (1, 3, 5, 10, 25)
n_subsets = {n_subsets_value}  # noqa: F821

# the value in megabytes to which the network should be throttled to
# (0.5, 1, 5, 10, 50, 100) megabytes
ui_network_throttling = {ui_network_throttling_value}  # noqa: F821

In [ ]:
### data setup

stellar_ra_dec = [
    (201.70022355, -47.47149884),
    (201.70063553, -47.47077343),
    (201.70112562, -47.4714947),
    (201.69904576, -47.47153201),
    (201.70102296, -47.47013743),
] * u.deg

stellar_ra_dec = stellar_ra_dec[:n_subsets]
aperture_radius = 0.3 * u.arcsec

# mean of the ICRS coords:
field_center = SkyCoord(*stellar_ra_dec.mean(0))
coords = SkyCoord(*stellar_ra_dec.T)


### functions
def user_default_view(viz, field_center):
    """
    user-defined "default" viewport settings
    """
    aid = viz.default_viewer._obj.glue_viewer.aid
    aid.set_viewport(center=field_center, fov=10 * u.arcsec)


def recenter_and_create_annuli(
    viz, subset_tools, dataset, aperture_radius, n_recenters=3
):
    annulus_min = aperture_radius
    annulus_max = 2 * aperture_radius
    refdata_wcs = viz.default_viewer._obj.glue_viewer.state.reference_data.coords
    aid = viz.default_viewer._obj.glue_viewer.aid

    for coord, subset_idx in zip(coords, range(1, n_subsets + 1)):
        subset_tools.subset = subset_tools.subset.choices[subset_idx]
        subset_tools.recenter_dataset = dataset
        aid.set_viewport(center=coord, fov=3 * u.arcsec)

        for _ in range(n_recenters):
            subset_tools.recenter()

        recentered_pixel_region = subset_tools.subset.selected_obj[0]["region"]
        recentered_sky_region = recentered_pixel_region.to_sky(refdata_wcs)

        bg_aperture = CircleAnnulusSkyRegion(
            recentered_sky_region.center, annulus_min, annulus_max
        )
        subset_tools.import_region(bg_aperture)

In [ ]:
### Initialize and show Imviz

# this block should be run once and cache the data for future use
viz = Imviz()
viz.load_data(
    "mast:jwst/product/jw04343002001_03101_00001_nrcblong_cal.fits",
    cache=True,
    gwcs_to_fits_sip=gwcs_to_fits_sip,
)
viz.load_data(
    "mast:jwst/product/jw04343001001_04101_00002_mirimage_cal.fits",
    cache=True,
    gwcs_to_fits_sip=gwcs_to_fits_sip,
)
orientation = viz.plugins["Orientation"]
orientation.align_by = "WCS"
viz.app.get_viewer("imviz-0")._composite_image.compression = image_compression
viz.show("sidecar:split-right", height=viewport_pixel_size)
# try with bottom, right should work!

In [ ]:
### We'll parse coordinates for five stars of interest, and center the viewer on the center of those coordinates.

# add markers at the initial centroids
viz.default_viewer.add_markers(Table({"coord": coords}), use_skycoord=True)

# set the viewport center and field of view:
user_default_view(viz, field_center)

In [ ]:
### Generate sky regions for apertures centered on each star, load those regions into jdaviz's Subset Tools plugin:

aperture_radius = 0.3 * u.arcsec
apertures = [CircleSkyRegion(coord, radius=aperture_radius) for coord in coords]

subset_tools = viz.plugins["Subset Tools"]
subset_tools.import_region(apertures)

# jdaviz image subsets can be iteratively centered on their center of light, to converge towards a source's centroid.

available_datasets = [data.label for data in viz.app.data_collection[:2]]

In [ ]:
# In this cell, we:
# (1) center the viewer on a source;
# (2) recenter three times;
# (3) create and load circular sky annuli that share their centers with the apertures.
# The annuli will be used to measure the sky background.

photometry = viz.plugins["Aperture Photometry"]
source_apertures = photometry.aperture.choices[:n_subsets]
background_apertures = photometry.aperture.choices[n_subsets:]

for dataset in available_datasets:
    # blink to the dataset
    viz.default_viewer.blink_once()

    # delete existing background annuli, before creating new ones
    if f"Subset {n_subsets + 1}" in subset_tools.subset.choices:
        for i in range(n_subsets + 1, n_subsets * 2 + 1):
            subset_tools.delete_subset(f"Subset {i}")

    # recenter the apertures for this dataset
    recenter_and_create_annuli(viz, subset_tools, dataset, aperture_radius)

    photometry.dataset = dataset

    for i, (aperture, bg_aperture) in enumerate(
        zip(source_apertures, background_apertures), start=1
    ):
        photometry.aperture = aperture
        photometry.background = bg_aperture
        photometry.current_plot_type = "Radial Profile"
        photometry.calculate_photometry()

user_default_view(viz, field_center)

In [ ]:
### Export a screenshot of the viewer

export = viz.plugins["Export"]
exported_screenshot_path = export.export("screenshot_image_viewer.png", overwrite=True)

In [ ]:
### Export screenshot of the Aperture Photometry plugin plot with the radial profile:

photometry.dataset = available_datasets[-1]
photometry.aperture = "Subset 1"
_ = photometry.calculate_photometry()

export.plugin_plot = "Aperture Photometry: plot"
export.export("screenshot_curve_of_growth.png", overwrite=True)

In [ ]:
### Export photometry results:

photometry.export_table("photometry_results.ecsv", overwrite=True)